In [9]:
get_ipython().system('pip install --upgrade google-generativeai httpx')
print("✅ `google-generativeai` and `httpx` have been upgraded to their latest versions.")

✅ `google-generativeai` and `httpx` have been upgraded to their latest versions.


In [20]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [21]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key
# --- Securely Configure Your API Key ---



print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


In [22]:
# --- Agent Definition ---

def create_day_trip_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
        instruction="""
        You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

        Your Mission:
        Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

        Guidelines:
        1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
        2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
        3. **Real-Time Focus**: Search for current operating hours and special events.
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
        """,
        tools=[google_search]
    )

day_trip_agent = create_day_trip_agent()
print(f"🧞 Agent '{day_trip_agent.name}' is created and ready for adventure!")

🧞 Agent 'day_trip_agent' is created and ready for adventure!


In [23]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [24]:
# --- Let's test the Day Trip Genie! ---

async def run_day_trip_genie():
    # Create a new, single-use session for this query
    day_trip_session = await session_service.create_session(
        app_name=day_trip_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Plan a relaxing and artsy day trip near Sunnyvale, CA. Keep it affordable!"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(day_trip_agent, query, day_trip_session, my_user_id)

await run_day_trip_genie()

🗣️ User Query: 'Plan a relaxing and artsy day trip near Sunnyvale, CA. Keep it affordable!'

🚀 Running query for agent: 'day_trip_agent' in session: 'c10ce76c-432a-42d7-9a4b-50229badd658'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Get ready for a day of artistic appreciation and tranquil moments near Sunnyvale, all while keeping your budget in mind! This itinerary focuses on free and affordable activities that will leave you feeling refreshed and inspired.

---

## Relaxing & Artsy Day Trip near Sunnyvale, CA (Affordable)

**Date:** Friday, December 27, 2025

### 🎨 Morning (10:30 AM - 1:00 PM): Immerse in Art at the Triton Museum

Start your day with a visit to the **Triton Museum of Art** in Santa Clara. This gem offers free admission and free parking, making it an excellent affordable choice for art lovers. The museum features diverse exhibitions of contemporary and historical art, often highlighting local and regional artists. Take 

Get ready for a day of artistic appreciation and tranquil moments near Sunnyvale, all while keeping your budget in mind! This itinerary focuses on free and affordable activities that will leave you feeling refreshed and inspired.

---

## Relaxing & Artsy Day Trip near Sunnyvale, CA (Affordable)

**Date:** Friday, December 27, 2025

### 🎨 Morning (10:30 AM - 1:00 PM): Immerse in Art at the Triton Museum

Start your day with a visit to the **Triton Museum of Art** in Santa Clara. This gem offers free admission and free parking, making it an excellent affordable choice for art lovers. The museum features diverse exhibitions of contemporary and historical art, often highlighting local and regional artists. Take your time browsing the galleries and enjoying the serene outdoor sculpture garden.

*   **Location:** 1505 Warburton Ave, Santa Clara, CA 95050
*   **Operating Hours:** Galleries are typically open Tuesday-Sunday, 11 AM - 4 PM during the academic year.
*   **Cost:** Free admission, free parking.

### 🍽️ Lunch (1:15 PM - 2:30 PM): Affordable Flavors in Palo Alto

Head over to nearby Palo Alto for an affordable and flavorful lunch. **Zareen's Palo Alto** is highly recommended for its delicious and budget-friendly Pakistani and Indian cuisine, offering dishes like biryanis, wraps, and spicy fries. It provides outdoor dining options and is known for its fresh ingredients.

*   **Location:** 261 University Ave, Palo Alto, CA 94301 (or check their other Palo Alto locations)
*   **Cost:** Affordable ($)

### 🌳 Afternoon (2:45 PM - 5:00 PM): Serenity at Sunnyvale Baylands Park

After lunch, unwind with a relaxing escape to **Sunnyvale Baylands Park**. This expansive park offers over 70 acres of developed parkland and 105 acres of seasonal wetlands, providing a perfect setting for a leisurely stroll or a quiet moment in nature. Enjoy the fresh air, watch for local wildlife, and take in the scenic views along the San Francisco Bay Trail. In December, there are no parking fees.

*   **Location:** 999 E Caribbean Dr, Sunnyvale, CA 94089
*   **Operating Hours:** Open 8 AM to one half hour after sunset.
*   **Cost:** Free (no parking fees in December).

### 🎶 Evening (6:00 PM - 7:30 PM): Community Song Circle

For a unique and artsy evening experience, join the **"COMMUNITY SONG CIRCLE OPEN TO ALL - COME SING WITH US!"** event. This free gathering, happening at 4:00 PM PST, offers a chance to engage with local music and community in a relaxed setting. It's a perfect way to wind down the day with some creative expression.

*   **Location:** Check specific details on Meetup.com for "Rhythms of Wellness - Cultivate Joy & Vitality thru Music!"
*   **Time:** 4:00 PM PST (check for exact duration)
*   **Cost:** Free.

### 🌙 Dinner (7:45 PM onwards): Cozy & Affordable Dinner

Conclude your day with a cozy and affordable dinner. Depending on your preference and proximity to the song circle, consider another affordable spot in Sunnyvale or Palo Alto. Many casual eateries and diverse cuisines can be found in downtown Sunnyvale or on California Avenue in Palo Alto. If you're still in Palo Alto, **Oren's Hummus** offers fresh, generous portions of hummus plates, shawarma wraps, and falafel bowls at reasonable prices.

--------------------------------------------------



In [25]:
# --- Tool Definition: A function that calls a live public API ---
import requests
import json

# A simple lookup to avoid needing a separate geocoding API for this example
LOCATION_COORDINATES = {
    "sunnyvale": "37.3688,-122.0363",
    "san francisco": "37.7749,-122.4194",
    "lake tahoe": "39.0968,-120.0324"
}

def get_live_weather_forecast(location: str) -> dict:
    """Gets the current, real-time weather forecast for a specified location in the US.

    Args:
        location: The city name, e.g., "San Francisco".

    Returns:
        A dictionary containing the temperature and a detailed forecast.
    """
    print(f"🛠️ TOOL CALLED: get_live_weather_forecast(location='{location}')")

    # Find coordinates for the location
    normalized_location = location.lower()
    coords_str = None
    for key, val in LOCATION_COORDINATES.items():
        if key in normalized_location:
            coords_str = val
            break
    if not coords_str:
        return {"status": "error", "message": f"I don't have coordinates for {location}."}

    try:
        # NWS API requires 2 steps: 1. Get the forecast URL from the coordinates.
        points_url = f"https://api.weather.gov/points/{coords_str}"
        headers = {"User-Agent": "ADK Example Notebook"}
        points_response = requests.get(points_url, headers=headers)
        points_response.raise_for_status() # Raise an exception for bad status codes
        forecast_url = points_response.json()['properties']['forecast']

        # 2. Get the actual forecast from the URL.
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()

        # Extract the relevant forecast details
        current_period = forecast_response.json()['properties']['periods'][0]
        return {
            "status": "success",
            "temperature": f"{current_period['temperature']}°{current_period['temperatureUnit']}",
            "forecast": current_period['detailedForecast']
        }
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Agent Definition: An agent that USES the new tool ---

weather_agent = Agent(
    name="weather_aware_planner",
    model="gemini-2.5-flash",
    description="A trip planner that checks the real-time weather before making suggestions.",
    instruction="You are a cautious trip planner. Before suggesting any outdoor activities, you MUST use the `get_live_weather_forecast` tool to check conditions. Incorporate the live weather details into your recommendation.",
    tools=[get_live_weather_forecast]
)

print(f"🌦️ Agent '{weather_agent.name}' is created and can now call a live weather API!")

🌦️ Agent 'weather_aware_planner' is created and can now call a live weather API!


In [26]:
# --- Let's test the Weather-Aware Planner ---

async def run_weather_planner_test():
    weather_session = await session_service.create_session(app_name=weather_agent.name, user_id=my_user_id)
    query = "I want to go hiking near Lake Tahoe, what's the weather like?"
    print(f"🗣️ User Query: '{query}'")
    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_planner_test()

🗣️ User Query: 'I want to go hiking near Lake Tahoe, what's the weather like?'

🚀 Running query for agent: 'weather_aware_planner' in session: '73b5d69c-eb52-48db-a4bb-bed00c88d45a'...


EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'location': 'Lake Tahoe'
        },
        id='adk-af8be4a2-4405-4623-aa66-3f55d424922f',
        name='get_live_weather_forecast'
      ),
      thought_signature=b'\n\x9c\x02\x01r\xc8\xda|\xcfe\xa7a\xab\x12\x95\x95\r\xe3\xf0\xb8\xbb\x13Z<$?\x12\xb9\x98\xc3)J\xfa\xbe:\x15\x9e\x93\x81\xe0\xad*\xc2Bg\x9aG /N9\x1bA\xd1\xcd_\xaf\xbc\xc7%\xaaJjj\x13\x981\xe6\x17r\xc0\x97\xbd\xd3\xbb\x1eN\x11\xee\xc6A\x0ekm\xebWI\x04\xb7\x07v\xd4\xb6[>\x0e\x97...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=20,
  prompt_token_count=187,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_c

The weather near Lake Tahoe is 30°F with areas of freezing fog and a 30% chance of snow. It will be mostly cloudy with a southwest wind around 10 mph.

Given these conditions, hiking might be challenging and potentially unsafe due to the freezing fog and chance of snow. If you do decide to go, please dress warmly, wear waterproof gear, and be aware of slippery conditions. It might be a good day to consider an alternative indoor activity or a less exposed trail.

--------------------------------------------------



In [27]:
import asyncio
from google.adk.tools import ToolContext
from google.adk.tools.agent_tool import AgentTool

# Assume 'db_agent' is a pre-defined NL2SQL Agent
# For this example, we'll create placeholder agents.

db_agent = Agent(
    name="db_agent",
    model="gemini-2.5-flash",
    instruction="You are a database agent. When asked for data, return this mock JSON object: {'status': 'success', 'data': [{'name': 'The Grand Hotel', 'rating': 5, 'reviews': 450}, {'name': 'Seaside Inn', 'rating': 4, 'reviews': 620}]}")

# --- 1. Define the Specialist Agents ---

# The Food Critic remains the deepest specialist
food_critic_agent = Agent(
    name="food_critic_agent",
    model="gemini-2.5-flash",
    instruction="You are a snobby but brilliant food critic. You ONLY respond with a single, witty restaurant suggestion near the provided location.",
)

# The Concierge knows how to use the Food Critic
concierge_agent = Agent(
    name="concierge_agent",
    model="gemini-2.5-flash",
    instruction="You are a five-star hotel concierge. If the user asks for a restaurant recommendation, you MUST use the `food_critic_agent` tool. Present the opinion to the user politely.",
    tools=[AgentTool(agent=food_critic_agent)]
)


# --- 2. Define the Tools for the Orchestrator ---

async def call_db_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    Use this tool FIRST to connect to the database and retrieve a list of places, like hotels or landmarks.
    """
    print("--- TOOL CALL: call_db_agent ---")
    agent_tool = AgentTool(agent=db_agent)
    db_agent_output = await agent_tool.run_async(
        args={"request": question}, tool_context=tool_context
    )
    # Store the retrieved data in the context's state
    tool_context.state["retrieved_data"] = db_agent_output
    return db_agent_output


async def call_concierge_agent(
    question: str,
    tool_context: ToolContext,
):
    """
    After getting data with call_db_agent, use this tool to get travel advice, opinions, or recommendations.
    """
    print("--- TOOL CALL: call_concierge_agent ---")
    # Retrieve the data fetched by the previous tool
    input_data = tool_context.state.get("retrieved_data", "No data found.")

    # Formulate a new prompt for the concierge, giving it the data context
    question_with_data = f"""
    Context: The database returned the following data: {input_data}

    User's Request: {question}
    """

    agent_tool = AgentTool(agent=concierge_agent)
    concierge_output = await agent_tool.run_async(
        args={"request": question_with_data}, tool_context=tool_context
    )
    return concierge_output


# --- 3. Define the Top-Level Orchestrator Agent ---

trip_data_concierge_agent = Agent(
    name="trip_data_concierge",
    model="gemini-2.5-flash",
    description="Top-level agent that queries a database for travel data, then calls a concierge agent for recommendations.",
    tools=[call_db_agent, call_concierge_agent],
    instruction="""
    You are a master travel planner who uses data to make recommendations.

    1.  **ALWAYS start with the `call_db_agent` tool** to fetch a list of places (like hotels) that match the user's criteria.

    2.  After you have the data, **use the `call_concierge_agent` tool** to answer any follow-up questions for recommendations, opinions, or advice related to the data you just found.
    """,
)

print(f"✅ Orchestrator Agent '{trip_data_concierge_agent.name}' is defined and ready.")

✅ Orchestrator Agent 'trip_data_concierge' is defined and ready.


In [28]:
# --- Let's test the Trip Data Concierge Agent ---

async def run_trip_data_concierge():
    """
    Sets up a session and runs a query against the top-level
    trip_data_concierge_agent.
    """
    # Create a new, single-use session for this query
    concierge_session = await session_service.create_session(
        app_name=trip_data_concierge_agent.name,
        user_id=my_user_id
    )

    # This query is specifically designed to trigger the full two-step process:
    # 1. Get data from the db_agent.
    # 2. Get a recommendation from the concierge_agent based on that data.
    query = "Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews."
    print(f"🗣️ User Query: '{query}'")

    # We call our existing helper function with the top-level orchestrator agent
    await run_agent_query(trip_data_concierge_agent, query, concierge_session, my_user_id)

# Run the test
await run_trip_data_concierge()

🗣️ User Query: 'Find the top-rated hotels in San Francisco from the database, then suggest a dinner spot near the one with the most reviews.'

🚀 Running query for agent: 'trip_data_concierge' in session: '5800790f-7903-4166-95d3-b3efdc950417'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'question': 'top-rated hotels in San Francisco'
        },
        id='adk-72b3cd87-6b7b-4056-bd6f-5a76b2f43507',
        name='call_db_agent'
      ),
      thought_signature=b'\n\xf2\x03\x01r\xc8\xda|\x07\xe0b\x8c\x04#0\xc4sr\xb4\xad\x08Y\xc8\xd5.\xce7\xb1\xe4G\x11\xd0`\xbd\xfc7\xf5\xa8\xf1"\xeb\xf1\x07\xe89\n\x104,\x86\xf7\x18\x07x\xbd\xabm\xfc^y\xc9\xdd/\xf3j\x92H;%\x8a\xfe\xa2\t\xed\xbe\'\xc20St\x82+p\xdd\xd1\xb0\xb7H\xfcY\xc6\x15C\xde/\xb2\xd3...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=Non

The top-rated hotels in San Francisco are:
*   **The Grand Hotel** (Rating: 5, Reviews: 450)
*   **Seaside Inn** (Rating: 4, Reviews: 620)

The hotel with the most reviews is the Seaside Inn. For a dinner spot near the Seaside Inn, our esteemed food critic suggests that "One might, if truly famished and lacking alternative, deign to enter *The Buoy & Grotto*."

--------------------------------------------------



In [30]:
# --- Agent Definition: The Adaptive Planner ---

def create_multi_day_trip_agent():
    """Create the Progressive Multi-Day Trip Planner agent"""
    return Agent(
        name="multi_day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent that progressively plans a multi-day trip, remembering previous days and adapting to user feedback.",
        instruction="""
        You are the "Adaptive Trip Planner" 🗺️ - an AI assistant that builds multi-day travel itineraries step-by-step.

        Your Defining Feature:
        You have short-term memory. You MUST refer back to our conversation to understand the trip's context, what has already been planned, and the user's preferences. If the user asks for a change, you must adapt the plan while keeping the unchanged parts consistent.

        Your Mission:
        1.  **Initiate**: Start by asking for the destination, trip duration, and interests.
        2.  **Plan Progressively**: Plan ONLY ONE DAY at a time. After presenting a plan, ask for confirmation.
        3.  **Handle Feedback**: If a user dislikes a suggestion (e.g., "I don't like museums"), acknowledge their feedback, and provide a *new, alternative* suggestion for that time slot that still fits the overall theme.
        4.  **Maintain Context**: For each new day, ensure the activities are unique and build logically on the previous days. Do not suggest the same things repeatedly.
        5.  **Final Output**: Return each day's itinerary in MARKDOWN format.
        """,
        tools=[google_search]
    )

multi_day_agent = create_multi_day_trip_agent()
print(f"🗺️ Agent '{multi_day_agent.name}' is created and ready to plan and adapt!")

🗺️ Agent 'multi_day_trip_agent' is created and ready to plan and adapt!


In [32]:
# --- Scenario 2: Testing Adaptation and Memory ---

async def run_adaptive_memory_demonstration():
    print("###  AGENT THAT ADAPTS (SAME SESSION) ###")

    # Create ONE session that we will reuse for the whole conversation
    trip_session = await session_service.create_session(
        app_name=multi_day_agent.name,
        user_id=my_user_id
    )
    print(f"Created a single session for our trip: {trip_session.id}")

    # --- Turn 1: The user initiates the trip ---
    query1 = "Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food."
    print(f"\n🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(multi_day_agent, query1, trip_session, my_user_id)

    # --- Turn 2: The user gives FEEDBACK and asks for a CHANGE ---
    # We use the EXACT SAME `trip_session` object!
    query2 = "That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?"
    print(f"\n🗣️ User (Turn 2 - Feedback): '{query2}'")
    await run_agent_query(multi_day_agent, query2, trip_session, my_user_id)

    # --- Turn 3: The user confirms and asks to continue ---
    query3 = "Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind."
    print(f"\n🗣️ User (Turn 3 - Confirmation): '{query3}'")
    await run_agent_query(multi_day_agent, query3, trip_session, my_user_id)

await run_adaptive_memory_demonstration()

###  AGENT THAT ADAPTS (SAME SESSION) ###
Created a single session for our trip: 3c4ee92b-ad9f-403b-8734-699371724c8a

🗣️ User (Turn 1): 'Hi! I want to plan a 2-day trip to Lisbon, Portugal. I'm interested in historic sites and great local food.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '3c4ee92b-ad9f-403b-8734-699371724c8a'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Great! Lisbon is a fantastic choice for history and food lovers. Let's start planning your first day.

Here's a possible itinerary for Day 1 in Lisbon, focusing on historic sites and delicious local cuisine:

**Day 1: Alfama and Baixa - Historic Charm and Culinary Delights**

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District & Lisbon Cathedral**
    *   Begin your day by getting lost in the narrow, winding streets of Alfama, Lisbon's oldest district. This area survived the 1755 earthquake and offers a glimpse into medieval Lisbon.
    *   Vis

Great! Lisbon is a fantastic choice for history and food lovers. Let's start planning your first day.

Here's a possible itinerary for Day 1 in Lisbon, focusing on historic sites and delicious local cuisine:

**Day 1: Alfama and Baixa - Historic Charm and Culinary Delights**

*   **Morning (9:00 AM - 1:00 PM): Explore Alfama District & Lisbon Cathedral**
    *   Begin your day by getting lost in the narrow, winding streets of Alfama, Lisbon's oldest district. This area survived the 1755 earthquake and offers a glimpse into medieval Lisbon.
    *   Visit the **Lisbon Cathedral (Sé de Lisboa)**, a majestic Romanesque cathedral with a rich history.
    *   Wander up to the **Miradouro das Portas do Sol** and **Miradouro de Santa Luzia** for breathtaking views over Alfama and the Tagus River.
*   **Lunch (1:00 PM - 2:30 PM): Traditional Portuguese Lunch in Alfama**
    *   Enjoy a traditional Portuguese lunch at a local tasca (tavern) in Alfama. Look for places serving "Bacalhau à Brás" (codfish with scrambled eggs and potatoes) or "Sardinhas Assadas" (grilled sardines, especially if you're there in summer).
*   **Afternoon (2:30 PM - 6:00 PM): Baixa District & Rossio Square**
    *   Head down to the Baixa district, rebuilt after the 1755 earthquake with a grid-like street plan.
    *   Walk along **Rua Augusta**, a pedestrian street leading to the impressive **Arco da Rua Augusta** and **Praça do Comércio** (Commerce Square).
    *   Explore **Rossio Square (Praça de D. Pedro IV)**, a lively hub in the city center with a distinctive wave-patterned pavement.
*   **Evening (7:30 PM onwards): Dinner & Fado in Alfama**
    *   Return to Alfama for a memorable dinner experience. Many restaurants in this area offer live **Fado music**, a soulful Portuguese musical genre, while you dine.

How does this sound for your first day? We can adjust anything you like!

--------------------------------------------------


🗣️ User (Turn 2 - Feedback): 'That sounds pretty good, but I'm not a huge fan of castles. Can you replace the morning activity for Day 1 with something else historical?'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '3c4ee92b-ad9f-403b-8734-699371724c8a'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""My apologies! I understand you'd prefer to skip the Lisbon Cathedral. Let's find a fantastic historical alternative for your morning.

How about we replace the morning activity for Day 1 with a visit to the historic Belém district? It's rich in history related to Portugal's Age of Discoveries and offers magnificent architecture that isn't a castle.

Here's the revised itinerary for Day 1:

**Day 1: Belém and Baixa - Age of Discoveries & City Center Charm**

*   **Morning (9:00 AM - 1:00 PM): Discover Belém - Age of Discoveries History**
    *   Start your day in Belém, a dis

My apologies! I understand you'd prefer to skip the Lisbon Cathedral. Let's find a fantastic historical alternative for your morning.

How about we replace the morning activity for Day 1 with a visit to the historic Belém district? It's rich in history related to Portugal's Age of Discoveries and offers magnificent architecture that isn't a castle.

Here's the revised itinerary for Day 1:

**Day 1: Belém and Baixa - Age of Discoveries & City Center Charm**

*   **Morning (9:00 AM - 1:00 PM): Discover Belém - Age of Discoveries History**
    *   Start your day in Belém, a district synonymous with Portugal's Age of Discoveries, easily accessible by tram or taxi.
    *   Visit the magnificent **Jerónimos Monastery (Mosteiro dos Jerónimos)**, a UNESCO World Heritage site and a prime example of Manueline architecture. Explore its stunning church and cloisters.
    *   Walk along the riverfront to see the iconic **Belém Tower (Torre de Belém)**, a 16th-century fortress that served as a point of embarkation and disembarkation for Portuguese explorers.
    *   Nearby, you can also see the **Monument to the Discoveries (Padrão dos Descobrimentos)**, celebrating Portugal's explorers.
*   **Lunch (1:00 PM - 2:30 PM): Iconic Treats and Lunch in Belém**
    *   No visit to Belém is complete without trying a warm, delicious "Pastel de Nata" (custard tart) from the original **Pastéis de Belém** bakery.
    *   Enjoy lunch at a local restaurant in Belém, perhaps trying some fresh seafood.
*   **Afternoon (2:30 PM - 6:00 PM): Baixa District & Rossio Square**
    *   After exploring Belém, take a tram or taxi back to the city center and head to the Baixa district, rebuilt after the 1755 earthquake with a grid-like street plan.
    *   Walk along **Rua Augusta**, a pedestrian street leading to the impressive **Arco da Rua Augusta** and **Praça do Comércio** (Commerce Square).
    *   Explore **Rossio Square (Praça de D. Pedro IV)**, a lively hub in the city center with a distinctive wave-patterned pavement.
*   **Evening (7:30 PM onwards): Dinner & Fado in Alfama**
    *   Head to the historic Alfama district for a memorable dinner experience. Many restaurants in this area offer live **Fado music**, a soulful Portuguese musical genre, while you dine.

How does this revised plan for Day 1 sound to you? We've swapped the cathedral for the impressive historical sites in Belém.

--------------------------------------------------


🗣️ User (Turn 3 - Confirmation): 'Yes, the new plan for Day 1 is perfect! Please plan Day 2 now, keeping the food theme in mind.'

🚀 Running query for agent: 'multi_day_trip_agent' in session: '3c4ee92b-ad9f-403b-8734-699371724c8a'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""Great! I'm glad Day 1 is perfect. Let's craft an equally engaging Day 2, keeping your love for history and local food firmly in mind.

Here's a proposed itinerary for your second day in Lisbon:

**Day 2: Chiado, Bairro Alto & Culinary Exploration**

*   **Morning (9:30 AM - 1:00 PM): Elegant Chiado and Historic Carmo Ruins**
    *   Start your day in **Chiado**, Lisbon's elegant and bohemian district, known for its beautiful squares, historic theaters, and traditional shops.
    *   Visit **A Brasileira Café**, one of Lisbon's oldest and most famous cafes, a perfect spot to enjoy a coffee and soak in the atmospher

Great! I'm glad Day 1 is perfect. Let's craft an equally engaging Day 2, keeping your love for history and local food firmly in mind.

Here's a proposed itinerary for your second day in Lisbon:

**Day 2: Chiado, Bairro Alto & Culinary Exploration**

*   **Morning (9:30 AM - 1:00 PM): Elegant Chiado and Historic Carmo Ruins**
    *   Start your day in **Chiado**, Lisbon's elegant and bohemian district, known for its beautiful squares, historic theaters, and traditional shops.
    *   Visit **A Brasileira Café**, one of Lisbon's oldest and most famous cafes, a perfect spot to enjoy a coffee and soak in the atmosphere.
    *   Explore the fascinating **Carmo Convent Ruins (Convento do Carmo)**, an open-air archaeological museum. This former church was largely destroyed in the 1755 earthquake, and its roofless naves offer a powerful historical experience.
    *   From Chiado, take a short walk or ride the historic **Bica Funicular** up to **Bairro Alto**. During the day, its narrow, charming streets are quiet, offering a different historical perspective before they come alive at night.
*   **Lunch (1:00 PM - 2:30 PM): Gastronomic Delights at Time Out Market (Mercado da Ribeira)**
    *   Head to the vibrant **Time Out Market (Mercado da Ribeira)** near Cais do Sodré. This bustling food hall is a foodie paradise, offering an extensive selection of high-quality Portuguese and international cuisine from renowned chefs and local vendors. It's the perfect place to sample a variety of dishes.
*   **Afternoon (2:30 PM - 6:00 PM): Panoramic Views & Tram 28 Ride**
    *   After lunch, consider taking a ride on the iconic **Santa Justa Lift** for stunning panoramic views over the Baixa district and beyond (if you didn't manage to get a good view of it on Day 1).
    *   Alternatively, experience a ride on **Tram 28**. This historic tram offers a fantastic and scenic tour through many of Lisbon's most picturesque and historic neighborhoods, including Graça and Alfama, providing a unique perspective of the city.
*   **Evening (7:30 PM onwards): Farewell Dinner - Authentic Portuguese Flavors**
    *   For your farewell dinner, immerse yourself in authentic Portuguese flavors. Consider exploring the restaurants in the historic **Mouraria** district, known for its traditional eateries and Fado houses. Alternatively, seek out a restaurant that specializes in specific Portuguese dishes you haven't tried yet, such as "Arroz de Marisco" (seafood rice) or a delicious grilled "Bife à Portuguesa" (Portuguese-style steak).

How does this plan for Day 2 sound for concluding your trip to Lisbon?

--------------------------------------------------

